In [9]:
import os
import wget
import object_detection
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from matplotlib import pyplot as plt
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
 }

files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

PREDATOR SENSE

VIDEO DETECTION

In [10]:
import cv2
import numpy as np
import tensorflow as tf
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import label_map_util
from twilio.rest import Client
import time
import os  # For reading environment variables

# Initialize a timer variable to track the duration
start_time = None
VIDEO_PATH = '3.mp4'

# Retrieve Twilio credentials from environment variables
account_sid = os.getenv('TWILIO_ACCOUNT_SID')
auth_token = os.getenv('TWILIO_AUTH_TOKEN')
twilio_phone_number = os.getenv('TWILIO_PHONE_NUMBER')
recipient_phone_number = os.getenv('RECIPIENT_PHONE_NUMBER')

client = Client(account_sid, auth_token)

# Load the label map
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

# Load the trained model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-21')).expect_partial()

def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# Initialize video capture
cap = cv2.VideoCapture(VIDEO_PATH)

alert_sent = False  # Flag to track if alert has been sent

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    image_np = np.array(frame)

    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes'] + label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.8,
                agnostic_mode=False)

    # Check if any detection surpasses a certain accuracy threshold
    if np.any(detections['detection_scores'] > 0.9):
        if not alert_sent:
            # Send message
            message = client.messages.create(
                body="WILD ANIMAL DETECTED!",
                from_=twilio_phone_number,
                to=recipient_phone_number)
            print("Message sent:", message.sid)#
            alert_sent = True  # Set alert_sent flag to True

    cv2.imshow('object detection', cv2.resize(image_np_with_detections, (800, 600)))

    if cv2.waitKey(10) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break


Message sent: SM11a21df16e305802a63420a03f701212


UIUX OF PREDATOR SENSE

In [3]:
import cv2
import numpy as np
import tensorflow as tf
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import label_map_util
from twilio.rest import Client
import time  # Import time module for timing
import os
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from PIL import Image, ImageTk
import threading  # Import threading for real-time detection

# Load the label map
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

# Load the trained model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-21')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# Initialize video capture
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

def start_detection(selected_animals):
    start_time = None
    alert_sent = False
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        image_np = np.array(frame)

        input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
        detections = detect_fn(input_tensor)

        num_detections = int(detections.pop('num_detections'))
        detections = {key: value[0, :num_detections].numpy()
                      for key, value in detections.items()}
        detections['num_detections'] = num_detections

        detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

        label_id_offset = 1
        image_np_with_detections = image_np.copy()

        viz_utils.visualize_boxes_and_labels_on_image_array(
                    image_np_with_detections,
                    detections['detection_boxes'],
                    detections['detection_classes'] + label_id_offset,
                    detections['detection_scores'],
                    category_index,
                    use_normalized_coordinates=True,
                    max_boxes_to_draw=5,
                    min_score_thresh=.8,
                    agnostic_mode=False)

        # Check if any detection surpasses a certain accuracy threshold
        if np.any(detections['detection_scores'] > 0.9):
            if start_time is None:
                start_time = time.time()  # Start the timer when the score threshold is met
            else:
                if time.time() - start_time >= 2:  # Check if 2 seconds have passed
                    if not alert_sent:
                        # Send message
                        # message = client.messages.create(
                        #     body="WILD ANIMAL DETECTED!",
                        #     from_=twilio_phone_number,
                        #     to=recipient_phone_number)
                        print("Message sent")  # Uncomment Twilio code if needed
                        alert_sent = True  # Set alert_sent flag to True
        else:
            start_time = None  # Reset the timer if score threshold is not met

        cv2.imshow('object detection', cv2.resize(image_np_with_detections, (800, 600)))

        if cv2.waitKey(10) & 0xFF == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            break

def start_detection_ui():
    selected_animals = {animal: var.get() for animal, var in animals.items()}
    if any(selected_animals.values()):
        # Run detection in a separate thread
        detection_thread = threading.Thread(target=start_detection, args=(selected_animals,))
        detection_thread.start()
    else:
        messagebox.showwarning("No Animal Selected", "Please select at least one animal.")

def stop_detection():
    print("Object detection stopped")

def exit_app():
    cap.release()
    cv2.destroyAllWindows()
    root.quit()

root = tk.Tk()
root.title("PredatorSense")
root.configure(bg="#D3D3D3")

animals = {"cat": tk.BooleanVar(), "dog": tk.BooleanVar(), "leopard": tk.BooleanVar(), "snake": tk.BooleanVar()}

def create_checkboxes():
    checkbox_frame = ttk.Frame(root, padding="10")
    checkbox_frame.pack(pady=20, padx=20, fill=tk.BOTH, expand=True)

    # Load and display the image
    image_path = "Untitled.jpeg"  # Replace with the actual path
    image = Image.open(image_path)
    image = image.resize((100, 100), Image.LANCZOS)  # Use Image.LANCZOS for antialiasing
    photo = ImageTk.PhotoImage(image)

    image_label = ttk.Label(checkbox_frame, image=photo, background='white')
    image_label.image = photo  # Keep a reference
    image_label.pack(side=tk.LEFT, padx=10)

    for animal, var in animals.items():
        chk_button = ttk.Checkbutton(checkbox_frame, text=animal, variable=var)
        chk_button.pack(pady=2, anchor=tk.W)

    start_button = ttk.Button(root, text="Start Detection", command=start_detection_ui)
    start_button.pack(pady=20, padx=40, ipadx=20, ipady=10)

    stop_button = ttk.Button(root, text="Stop Detection", command=stop_detection)
    stop_button.pack(pady=20, padx=40, ipadx=20, ipady=10)

    exit_button = ttk.Button(root, text="Exit", command=exit_app)
    exit_button.pack(pady=20, padx=40, ipadx=20, ipady=10)

create_checkboxes()
root.mainloop()
